In [1]:
from l2r_class import *

### Chargement json (Top documents les plus non-pertinents pour chaque requete selon le score BM25)

In [2]:
import json

# Load the JSON data from a file
data_list = []
with open('msmarco-hard-negatives.jsonl', 'r') as file:
	for line in file:
		print(json.loads(line))
		data_list.append(json.loads(line))
		if len(data_list) > 3:
			break

{'qid': '571018', 'pos': [{'pid': '7349777', 'ce-score': 10.257502555847168}], 'neg': {'bm25': [{'pid': '6948601', 'bm25-score': 26.068258, 'ce-score': 3.581289291381836}, {'pid': '5129919', 'bm25-score': 26.007563, 'ce-score': 8.257364273071289}, {'pid': '6717931', 'bm25-score': 25.934547, 'ce-score': 8.866464614868164}, {'pid': '1065943', 'bm25-score': 25.258688, 'ce-score': 5.258519172668457}, {'pid': '1626276', 'bm25-score': 25.153275, 'ce-score': 4.19371223449707}, {'pid': '981824', 'bm25-score': 24.716213, 'ce-score': 8.563857078552246}, {'pid': '6449111', 'bm25-score': 24.62115, 'ce-score': 4.907355785369873}, {'pid': '1028927', 'bm25-score': 24.54112, 'ce-score': 7.617893695831299}, {'pid': '2524942', 'bm25-score': 24.355902, 'ce-score': 1.5268436670303345}, {'pid': '5810175', 'bm25-score': 24.315733, 'ce-score': -0.6152520179748535}, {'pid': '6236527', 'bm25-score': 24.3093, 'ce-score': -2.9456772804260254}, {'pid': '7179545', 'bm25-score': 24.130537, 'ce-score': 10.0183410644

In [3]:
def create_triplet(data_list):
	# Now data_list contains multiple JSON objects, one per line from the file
	data_id = []

	for data in data_list:
		# Extract positive examples
		pos_examples = [(data['qid'], pos['pid'], 1) for i, pos in enumerate(data['pos']) if i < 1]

		# Extract negative examples with a bm25_score
		neg_examples = [(data['qid'], neg['pid'], 0) for i, neg in enumerate(data['neg']['bm25']) if i < 9]

		# Combine positive and negative examples
		data_id.extend(pos_examples + neg_examples)

	# Print the final_data
	print("data size", len(data_id))
	print(data_id[:20])
	return data_id

data_id = create_triplet(data_list)

data size 40
[('571018', '7349777', 1), ('571018', '6948601', 0), ('571018', '5129919', 0), ('571018', '6717931', 0), ('571018', '1065943', 0), ('571018', '1626276', 0), ('571018', '981824', 0), ('571018', '6449111', 0), ('571018', '1028927', 0), ('571018', '2524942', 0), ('832106', '5090093', 1), ('832106', '5090096', 0), ('832106', '210166', 0), ('832106', '4877671', 0), ('832106', '851808', 0), ('832106', '210158', 0), ('832106', '2492227', 0), ('832106', '3094439', 0), ('832106', '5434531', 0), ('832106', '210165', 0)]


In [4]:
def create_tripletWSLS(data_list):
	# Now data_list contains multiple JSON objects, one per line from the file
	data_id = []
	bm25_max = 0
	neg_examples = []

	for data in data_list:
		# Extract positive examples
		pos_examples = [(data['qid'], pos['pid'], 1) for i, pos in enumerate(data['pos']) if i < 1]

		# Extract negative examples with a bm25_score
		for i, neg in enumerate(data['neg']['bm25']):
			if i < 9:
				if neg['bm25-score'] > bm25_max:
					bm25_max = neg['bm25-score'] + 1
				neg_examples.append((data['qid'], neg['pid'], neg['bm25-score']))
		# Combine positive and negative examples
		data_id.extend(pos_examples)

	print(neg_examples[0])
	print(bm25_max)
	neg_examples = [(qid, pid, float(score) / float(bm25_max)) for qid, pid, score in neg_examples]
	data_id.extend(neg_examples)

	# Print the final_data
	print("data size", len(data_id))
	print("bm25_max : ", bm25_max)
	print(data_id[:20])
	return data_id

data_id_wsls = create_tripletWSLS(data_list)

('571018', '6948601', 26.068258)
43.982548
data size 40
bm25_max :  43.982548
[('571018', '7349777', 1), ('832106', '5090093', 1), ('777651', '2439196', 1), ('174637', '6005238', 1), ('571018', '6948601', 0.5926954936762645), ('571018', '5129919', 0.5913155145081636), ('571018', '6717931', 0.5896554015015227), ('571018', '1065943', 0.5742888747600525), ('571018', '1626276', 0.5718921741414345), ('571018', '981824', 0.5619550054262431), ('571018', '6449111', 0.5597936254170631), ('571018', '1028927', 0.5579740400669829), ('571018', '2524942', 0.5537628697637071), ('832106', '5090096', 0.9772637092330348), ('832106', '210166', 0.8639415797374903), ('832106', '4877671', 0.8277626389448833), ('832106', '851808', 0.8244742437386756), ('832106', '210158', 0.7357577373643747), ('832106', '2492227', 0.7230036559046101), ('832106', '3094439', 0.6955360567104935)]


In [5]:
import ir_datasets

# Load the msmarco dataset
dataset = ir_datasets.load('msmarco-passage/train')
store = dataset.docs_store()
print(dataset)

Dataset(id='msmarco-passage/train', provides=['docs', 'queries', 'qrels', 'scoreddocs', 'docpairs'])


In [6]:
dict_query = {}

for query in dataset.queries_iter():
	dict_query[query.query_id] = query.text
	

### Create NSBM25

In [7]:
import numpy as np

def create_nsbm25():
	# Create an empty list to store the modified data
	data_bm25 = []

	for data in data_id:
		# Extract the values from the original data
		dict_value = dict_query[data[0]]
		store_value = store.get(data[1])[1]
		class_value = data[2]
		
		# Perform one-hot encoding
		class_vector = np.zeros(2)
		class_vector[class_value] = 1
		
		# Create a modified tuple with the encoded class vector
		modified_tuple = (dict_value, store_value, class_vector)
		
		# Append the modified tuple to the list
		data_bm25.append(modified_tuple)
	return data_bm25

data_bm25 = create_nsbm25()
# Print the modified_data
print(len(data_bm25))
print(data_bm25[:5])

40
[('what are the liberal arts?', 'liberal arts. 1. the academic course of instruction at a college intended to provide general knowledge and comprising the arts, humanities, natural sciences, and social sciences, as opposed to professional or technical subjects.', array([0., 1.])), ('what are the liberal arts?', 'The New York State Education Department requires 60 Liberal Arts credits in a Bachelor of Science program and 90 Liberal Arts credits in a Bachelor of Arts program. In the list of course descriptions, courses which are liberal arts for all students are identified by (Liberal Arts) after the course number.', array([1., 0.])), ('what are the liberal arts?', "What Does it Mean to Study Liberal Arts? A liberal arts major offers a broad overview of the arts, sciences, and humanities. Within the context of a liberal arts degree, you can study modern languages, music, English, anthropology, history, women's studies, psychology, math, political science or many other disciplines.", a

### Chargement modele BERT

In [8]:
tokenizer = BertTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
bert_model = BertModel.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'fit_denses.4.weight', 'fit_denses.4.bias', 'fit_denses.3.weight', 'fit_denses.1.bias', 'fit_denses.0.bias', 'fit_denses.0.weight', 'fit_denses.3.bias', 'fit_denses.1.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'fit_denses.2.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'fit_denses.2.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

### LS and T-LS

In [12]:
def learnModelLS(data, num_epochs=1, tls=-1, eps=1e-8):
	dataset = LTRDataset(data, tokenizer, max_len=512)

	# Create data loaders
	loader = DataLoader(dataset, batch_size=16, num_workers=4)

	# Instantiate the model
	model = BertLTRModel(bert_model)

	# Move model to GPU if available
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	model = model.to(device)

	# Define loss function and optimizer
	criterion = nn.CrossEntropyLoss(label_smoothing=eps)
	optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

	list_loss = []
	# Training loop
	for epoch in range(num_epochs):
		model.train()
		total_loss = 0

		for batch in loader:
			# Move batch tensors to the same device as the model 
			input_ids = batch['input_ids'].to(device)
			attention_mask = batch['attention_mask'].to(device)
			relevance = batch['relevance'].to(device)

			# Forward pass
			outputs = model(input_ids=input_ids, attention_mask=attention_mask)

			# Compute loss
			if tls == num_epochs:
				criterion = nn.CrossEntropyLoss()
			loss = criterion(outputs, relevance) 

			# Backward pass and optimize
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			total_loss += loss.item()
		loss = total_loss/len(loader)
		list_loss.append(loss)
		print(f'Epoch {epoch}, Loss: {loss}')
	return model, list_loss
	

### Creation Data_NSrandom

In [10]:
import random

def create_nsrandom_samples(data_bm25):
	data_nsrand = []
	queries = [data_bm25[i] for i in range(0, len(data_bm25), 10)]
	docs = {data_bm25[i][1] for i in range(len(data_bm25))}

	for query in queries:
		data_nsrand.append(query)
		chosen_docs = [query[1]]
		for i in range(9):
			doc = random.choice(list(docs))
			while doc in chosen_docs:
				doc = random.choice(list(docs))
			data_nsrand.append((query[0], doc, np.array([1., 0.])))
			chosen_docs.append(doc)
	return data_nsrand

In [11]:
data_nsrand = create_nsrandom_samples(data_bm25)
print(data_nsrand[:10])
print(data_bm25[:10])

[('what are the liberal arts?', 'liberal arts. 1. the academic course of instruction at a college intended to provide general knowledge and comprising the arts, humanities, natural sciences, and social sciences, as opposed to professional or technical subjects.', array([0., 1.])), ('what are the liberal arts?', 'Fibrinolytic therapy, also sometimes called “thrombolytic therapy,” is the use of special drugs to break up blood clots that are blocking a person or animal’s major artery.They are most commonly given to people after a heart attack or when a major blood clot has been discovered in a medical scan.ibrinolytic therapy is often administered as a drug. Fibrinolytic therapy is the use of special drugs to break up blood clots. Blood clots can lead to death if one gets into the heart and other vital organs.', array([1., 0.])), ('what are the liberal arts?', 'Electronic copy available at: http://ssrn.com/abstract=1473046 1 EXPLAINING THE RIDDLE OF DOLLAR COST AVERAGING Abstract: Dollar 

In [13]:
# LS
print('-----LS pour NS_BM25-----')
model_LS_bm25, list_loss_LS_bm25 = learnModelLS(data_bm25, num_epochs=5)
print('-----LS pour NS_rand-----')
model_LS_random, list_loss_LS_random = learnModelLS(data_nsrand, num_epochs=5)
# 10 minutes au total pour 5 queries...

-----LS pour NS_BM25-----
Epoch 0, Loss: 0.6750838359196981
Epoch 1, Loss: 0.6554948886235555
Epoch 2, Loss: 0.6359568039576212
Epoch 3, Loss: 0.6158643166224161
Epoch 4, Loss: 0.5957018733024597
-----LS pour NS_rand-----
Epoch 0, Loss: 0.6714686155319214


In [ ]:
# # TLS
# print('-----TLS pour NS_BM25-----')
# model_TLS_bm25, list_loss_TLS_bm25 = learnModelLS(data_bm25, num_epochs=5, tls=2)
# print('-----TLS pour NS_rand-----')
# model_TLS_random, list_loss_TLS_random = learnModelLS(data_nsrand, num_epochs=5, tls=2)
# # 11 minutes au total pour 5 queries...

In [ ]:
from wsls_class import *

def learnModelWSLS(data, num_epochs=1, tls=-1, eps=1e-8):
	dataset = LTRDataset(data, tokenizer, max_len=512)

	# Create data loaders
	loader = DataLoader(dataset, batch_size=16, num_workers=4)

	# Instantiate the model
	model = BertLTRModel(bert_model)

	# Move model to GPU if available
	device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
	model = model.to(device)

	# Define loss function and optimizer
	criterion = CrossEntropyLoss()
	optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

	list_loss = []
	# Training loop
	for epoch in range(num_epochs):
		model.train()
		total_loss = 0

		for batch in loader:
			# Move batch tensors to the same device as the model 
			input_ids = batch['input_ids'].to(device)
			attention_mask = batch['attention_mask'].to(device)
			relevance = batch['relevance'].to(device)

			# Forward pass
			outputs = model(input_ids=input_ids, attention_mask=attention_mask)

			# Compute loss
			if tls == num_epochs:
				criterion = nn.CrossEntropyLoss()
			loss = criterion(outputs, relevance) 

			# Backward pass and optimize
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

			total_loss += loss.item()
		loss = total_loss/len(loader)
		list_loss.append(loss)
		print(f'Epoch {epoch}, Loss: {loss}')
	return model, list_loss

In [ ]:
# LS
print('-----WSLS pour NS_BM25-----')
model_LS_bm25, list_loss_LS_bm25 = learnModelWSLS(data_bm25, num_epochs=5)
print('-----T-WSLS pour NSBM25-----')
model_LS_random, list_loss_LS_random = learnModelWSLS(data_nsrand, num_epochs=5)
# 10 minutes au total pour 5 queries...

### Metrique d'evaluation (Standard Deviation)